## Problem Set 4: Determinize NFA


_csc427, semester 212
<br>
university of miami
<br>
date: 17 february 2021
<br>
update: 23 february 2021_

---

### Student name:  Temuulen Ganbold csc427 ps4

---

## DFA Implementation


The MachineModel instantiates a DFA around a machine description. Its compute methond takes a string and returns whether that string is in (True) or not in (False) the language recognized by the DFA. The TestMachine class takes a machine description and a test vector and confirms or not if the test is passed.



In [8]:
"""
The verbose switch:
    Set this true or false, to run code verbosely
"""

verbose = True


In [9]:
class MachineModel:
    """
    A machine description is a dictionary with,
        'states': a list of states.
        'alphabet': a list of letters (strings of length one)
        'transitions': a dictionary with keys tuples (a state,a letter) to a state
        'start': a state (the start state)
        'accept': a list of states (the accepting states)
        
    The states are any hashable, and we use:
    - strings for simple DFA's, 
    - tuples for product DFA's, 
    - and frozensets for DFA's resulting from determinizing an NFA.
        
    """
    
    def __init__(self,machine_description):
        self.states = machine_description['states']
        self.alphabet = machine_description['alphabet']
        self.transitions = machine_description['transitions']
        self.start_state = machine_description['start'] 
        self.accept_states = machine_description['accept']
        self.current_state = self.start_state 

    def do_transition(self,letter):
        self.current_state = self.transitions[(self.current_state,letter)]
    
    def compute(self,word):
        self.current_state = self.start_state
        if verbose : print(self.current_state)
        for w in word:
            self.do_transition(w)
            if verbose : print(w,self.current_state)
        return self.current_state in self.accept_states

    def describe(self,name=""):
        print("Machine Description:",name)
        print("\tstates:",len(self.states))
        for s in self.states:
            print("\t\t",s)
        print("\ttransitions:",len(self.transitions))
        for t,v in self.transitions.items():
            print(f"\t\t{t}  ->  {v}")
        print("\taccept states:",len(self.accept_states))
        for a in self.accept_states:
            print("\t\t",a)
        print()

        
def test_machine(dfa_description,test_cases,name=""):
    
    print('running tests ...')
    dfa = MachineModel(dfa_description)
    if verbose: dfa.describe(name)
    for (t,r) in (test_cases):
        if dfa.compute(t) != r:
            print(r,'\t|'+t+'|','\tWRONG, ABORT')
            return False
        print(r,'\t|'+t+'|','\tOK')
    return True
 

## The Determinize Code

The objective is to show that any language recognized by an NFA can be recognized by a DFA. The proof is construcutive. Give that the language can be recognized by an NFA, we ask for such an NFA. From its description, we write the description of a DFA that accepts the same language.

The idea is revisit our NFA code, which evalutes all possible computation paths in parallel. At each step, we have a set of states. For each state in the set, there is a possible computation path to that state, at this point in the computation. We make a slight change of perspective. We consider that set to be "the" state, rather than a set of states. 

This change only requires that redefine the transition function to include as one step the many steps we undertook to advance the multiple computation paths. Essentially, we precompute the outcome of seeing a certain letter when to each state in a set of states, and simply see it is a single transition under that letter, from state set to state set.



In [10]:

class DeterminizeNFA:
    
    """   
    self.states is of type list(frozenset)
    self.alphabet is of type list(string)
    self.transitions is of type tuple(frozenset,string):frozenset
    self.start is of type frozenset
    self.accept is of type list(frozenset)
    
    an nfa_transitions variable is of the type of transitions for the 
    NFA machine: tuple(string,string):list(string)

    """
    
    def __init__(self,nfa_d):
        self.nfa_d = nfa_d
        self.states = [] 
        self.alphabet = []
        self.transitions = {}
        self.start = None
        self.accept = []


    def epsilon_one_step(self,state_set,nfa_transitions):
        """
        returns a frozenset representing the states in state_set
        or one epsilon edge away
        """
        
       
        ep = frozenset() 
        for state in state_set:
            t = (state,':')
            if t in nfa_transitions:
                ep = ep.union(nfa_transitions[t])           
        return ep.union(state_set)
    

    
    def epsilon_close(self,state_set,nfa_transitions):
        """
        returns a frozenset representing the state
        that is the epsilon closure of the given state_set
        """
        le = self.epsilon_one_step(state_set, nfa_transitions) 
        l = -1
        while(l < len(le) ):
            l = len(le)
            le = self.epsilon_one_step(le, nfa_transitions)
        return le 

    def do_transition(self, state_set, letter, nfa_transitions):
        """
        returns a frozenset representing state transitioned 
        from state_set on letter.
        """
        new = frozenset()
        for state in state_set:
            if (state, letter) in nfa_transitions:
                new = new.union(nfa_transitions[(state,letter)])

        return self.epsilon_close(new,nfa_transitions)

    def transform_start(self,nfa_start,nfa_transitions):
        """
        from nfa_start calculates self.start, as well as 
        adds the start state to self.starts
        """
        s = self.epsilon_close([nfa_start],nfa_transitions)
        self.start = s
        self.states.append(s)
        return s
    
    def transform_transitions(self,nfa_transitions):
        """
        from nfa_transitions, creates self.transitions,
        as well as adding all discovered states to self.states.
        """

        for st in self.states:
            for let in self.alphabet:
                s = self.do_transition( st, let , nfa_transitions)
                if s not in self.states:
                    self.states.append(s)
                self.transitions[(st,let)] = s


    
    def transform_accept(self,nfa_accept):
        """
        from nfa_accept, creates self.accept
        """
        for cur_acc in self.states:
            if(cur_acc.intersection(nfa_accept) ):
                self.accept.append(cur_acc)


    def transform(self):
        
        self.alphabet = self.nfa_d['alphabet'][:] # copy the list
        s_s = self.transform_start(self.nfa_d['start'],self.nfa_d['transitions'])
        self.transform_transitions(self.nfa_d['transitions'])
        self.transform_accept(self.nfa_d['accept'])

        return {
            'states':self.states,
            'alphabet':self.alphabet,
            'transitions':self.transitions,
            'start':self.start,
            'accept':self.accept
        }


### Simple test cases

In [11]:
# string ends with a b

T0 = {
    'states':['Q1','Q2'],
    'alphabet':['a','b'],
    'transitions':{
        ('Q1','a'):['Q1'], ('Q1','b'):['Q1','Q2'], 
    },
    'start':'Q1',
    'accept':['Q2']
}

test_cases = [
    ('b',True),('ab',True),('bb',True),
    ('',False),('a',False),('aa',False),
    ('ba',False)
]


verbose = True
dfa_desc = DeterminizeNFA(T0).transform()
res = test_machine(dfa_desc,test_cases,name="T0")
print('test result:', res)
print()

# accepts only the empty string
T1 = {
    'states':['Q1','Q2'],
    'alphabet':['a','b'],
    'transitions':{
        ('Q2','a'):['Q2'], ('Q2','b'):['Q2'],('Q1',':'):['Q2'] 
    },
    'start':'Q1',
    'accept':['Q1']
}

test_cases = [
    ('b',False),('ab',False),('bb',False),
    ('',True),('a',False),('aa',False),
    ('ba',False)
]

dfa_desc = DeterminizeNFA(T1).transform()
res = test_machine(dfa_desc,test_cases,name="T1")
print('test result:', res)
print()


running tests ...
Machine Description: T0
	states: 2
		 frozenset({'Q1'})
		 frozenset({'Q2', 'Q1'})
	transitions: 4
		(frozenset({'Q1'}), 'a')  ->  frozenset({'Q1'})
		(frozenset({'Q1'}), 'b')  ->  frozenset({'Q2', 'Q1'})
		(frozenset({'Q2', 'Q1'}), 'a')  ->  frozenset({'Q1'})
		(frozenset({'Q2', 'Q1'}), 'b')  ->  frozenset({'Q2', 'Q1'})
	accept states: 1
		 frozenset({'Q2', 'Q1'})

frozenset({'Q1'})
b frozenset({'Q2', 'Q1'})
True 	|b| 	OK
frozenset({'Q1'})
a frozenset({'Q1'})
b frozenset({'Q2', 'Q1'})
True 	|ab| 	OK
frozenset({'Q1'})
b frozenset({'Q2', 'Q1'})
b frozenset({'Q2', 'Q1'})
True 	|bb| 	OK
frozenset({'Q1'})
False 	|| 	OK
frozenset({'Q1'})
a frozenset({'Q1'})
False 	|a| 	OK
frozenset({'Q1'})
a frozenset({'Q1'})
a frozenset({'Q1'})
False 	|aa| 	OK
frozenset({'Q1'})
b frozenset({'Q2', 'Q1'})
a frozenset({'Q1'})
False 	|ba| 	OK
test result: True

running tests ...
Machine Description: T1
	states: 2
		 frozenset({'Q2', 'Q1'})
		 frozenset({'Q2'})
	transitions: 4
		(frozenset({'Q

### Basic Tests

In [12]:
 
def basic_test_determinize(nfa_l, test_cases_l):
    
    assert(len(nfa_l)==len(test_cases_l))
    
    correct = 0
    num_tests = len(test_cases_l)
    print(f"\n*** Running Basic Tests on {num_tests} machines")
    for i in range(num_tests):
        print("\nExercise",i)
        dfa = DeterminizeNFA(nfa_l[i]).transform()
        if test_machine(dfa,test_cases_l[i],name="machine "+str(i)):
            correct += 1
    print("\n*** correct:",correct,"out of",num_tests)
    if correct==num_tests:
        print("*** passsed")
    else:
        print("*** failed")


In [13]:

# testing
# these are basic tests only. 

nfa_eb = [None for i in range(6)]


# a ( b a )* b
nfa_eb[0] = {
    'states': ['S0','S1','B0','B1','BA0','BA1','BA2'],
    'alphabet': ['a','b'],
    'transitions': {
        ('S0','a'):['S1'],('S1',':'):['BA0','B0'],
        ('BA0','b'):['BA1'],('BA1','a'):['BA2'],
        ('BA2',':'):['BA0','B0'],
        ('B0','b'):['B1']
    },
    'start': 'S0',
    'accept': ['B1']
}

# ( : union a ) b
nfa_eb[1] = {
    'states': ['S','A0','A1','B'],
    'alphabet': ['a','b'],
    'transitions': {
        ('S',':'):['A1','A0'],
        ('A0','a'):['A1'],
        ('A1','b'):['B']
    },
    'start': 'S',
    'accept': ['B']
}

# sigma* a sigma* b sigma* a sigma* 
nfa_eb[2] = {
    'states': ['S0','S1','S2','S3'],
    'alphabet': ['a','b'],
    'transitions': {
        ('S0','a'):['S0','S1'],('S0','b'):['S0'],
        ('S1','a'):['S1'],('S1','b'):['S1','S2'],
        ('S2','a'):['S2','S3'],('S2','b'):['S2'],
        ('S3','a'):['S3'],('S3','b'):['S3']
    },
    'start': 'S0',
    'accept': ['S3']
}
 
# ( a union ba union bb) sigma*
nfa_eb[3] = {
    'states': ['S','A','BA0','BA1','BB0','BB1','S0','S1'],
    'alphabet': ['a','b'],
    'transitions': {
        ('S',':'):['A','BA0','BB0'],('A','a'):['S0'],
        ('BA0','b'):['BA1'],('BA1','a'):['S0'],
        ('BB0','b'):['BB1'],('BB1','b'):['S0'],
        ('S0',':'):['S1'],('S1','a'):['S1'],('S1','b'):['S1']
    },
    'start': 'S',
    'accept': ['S1']
}

# ( a union b)* aaa ( a union b)*
nfa_eb[4] = {
    'states': ['S0','S1','S2','S3','A1','A2'],
    'alphabet': ['a','b'],
    'transitions': {
        ('S0','a'):['S0'],('S0','b'):['S0'],('S0',':'):['S1'],
        ('S1','a'):['A1'],('A1','a'):['A2'],('A2','a'):['S2'],
        ('S2',':'):['S3'],
        ('S3','a'):['S3'],('S3','b'):['S3'],
    },
    'start': 'S0',
    'accept': ['S3']
}

# ( ((aa)* bb ) union ab )*
nfa_eb[5] = {
    'states': ['S0','S1','S2','A','B','C'],
    'alphabet': ['a','b'],
    'transitions': {
        ('S0',':'):['S1'],('S0','a'):['C'],('C','b'):['S2'],
        ('S1','a'):['A'],('A','a'):['S1'],
        ('S1','b'):['B'],('B','b'):['S2'],
        ('S2',':'):['S0']
    },
    'start': 'S0',
    'accept': ['S0','S2']
}

nfa_test = [None for i in range(6)]

nfa_test[0] = [
    ('b',False),
    ('ab',True),
    ('abab',True)
]
nfa_test[1] = [
    ('b',True),
    ('ab',True),
    ('a',False),
    ('abb',False)
]
nfa_test[2] = [
    ('aba',True),
    ('a',False),
    ('aabb',False),
    ('bbabab',True)
]
nfa_test[3] = [
    ('a',True),
    ('ba',True),
    ('bb',True),
    ('b',False)
]
nfa_test[4] = [
    ('aaa',True),
    ('aba',False)    
]
nfa_test[5] = [
    ('',True),
    ('bb',True),
    ('abaa',False)
]


In [14]:
basic_test_determinize(nfa_eb, nfa_test)


*** Running Basic Tests on 6 machines

Exercise 0
running tests ...
Machine Description: machine 0
	states: 5
		 frozenset({'S0'})
		 frozenset({'BA0', 'B0', 'S1'})
		 frozenset()
		 frozenset({'BA1', 'B1'})
		 frozenset({'BA0', 'BA2', 'B0'})
	transitions: 10
		(frozenset({'S0'}), 'a')  ->  frozenset({'BA0', 'B0', 'S1'})
		(frozenset({'S0'}), 'b')  ->  frozenset()
		(frozenset({'BA0', 'B0', 'S1'}), 'a')  ->  frozenset()
		(frozenset({'BA0', 'B0', 'S1'}), 'b')  ->  frozenset({'BA1', 'B1'})
		(frozenset(), 'a')  ->  frozenset()
		(frozenset(), 'b')  ->  frozenset()
		(frozenset({'BA1', 'B1'}), 'a')  ->  frozenset({'BA0', 'BA2', 'B0'})
		(frozenset({'BA1', 'B1'}), 'b')  ->  frozenset()
		(frozenset({'BA0', 'BA2', 'B0'}), 'a')  ->  frozenset()
		(frozenset({'BA0', 'BA2', 'B0'}), 'b')  ->  frozenset({'BA1', 'B1'})
	accept states: 1
		 frozenset({'BA1', 'B1'})

frozenset({'S0'})
b frozenset()
False 	|b| 	OK
frozenset({'S0'})
a frozenset({'BA0', 'B0', 'S1'})
b frozenset({'BA1', 'B1'})
True 	